In [25]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from datasets import load_dataset
import torch
import os
import torch
import torchaudio
import torchaudio.functional as F


In [26]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
print(torch.cuda.is_available())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

True


In [27]:
ds = load_dataset("MegaKosT/RuDevSberDS")

Using custom data configuration MegaKosT--RuDevSberDS-c815bd8b35ec831d
Found cached dataset parquet (/root/.cache/huggingface/datasets/MegaKosT___parquet/MegaKosT--RuDevSberDS-c815bd8b35ec831d/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 3/3 [00:00<00:00, 296.84it/s]


In [28]:
len(ds['train'])

43576

In [29]:
import datasets
datasets.__version__

'2.9.0'

In [30]:
print(torch.__version__)
print(torchaudio.__version__)

1.13.1+cu117
0.13.1+cu117


In [31]:
ds['train'][0]['audio']

{'path': '00003ec0-1257-42d1-b475-db1cd548092e.wav',
 'array': array([ 0.00787354,  0.00735474,  0.00714111, ..., -0.00018311,
        -0.00015259, -0.00018311]),
 'sampling_rate': 16000}

In [34]:
ds_train = ds['train'] #.select(range(5000))
ds_test = ds['test'] #.select(range(1500))
# len(ds_train)

In [35]:
ds_train[0]['audio']

{'path': '00003ec0-1257-42d1-b475-db1cd548092e.wav',
 'array': array([ 0.00787354,  0.00735474,  0.00714111, ..., -0.00018311,
        -0.00015259, -0.00018311]),
 'sampling_rate': 16000}

In [36]:
ds_train = ds_train.filter(
    lambda it1: (it1["transcription"] is not None) and (len(it1["transcription"].strip()) > 0)
)
ds_test = ds_test.filter(
    lambda it1: (it1["transcription"] is not None) and (len(it1["transcription"].strip()) > 0)
)

100%|██████████| 10/10 [00:04<00:00,  2.28ba/s]


In [37]:
processor = Wav2Vec2Processor.from_pretrained("bond005/wav2vec2-large-ru-golos")
model = Wav2Vec2ForCTC.from_pretrained("bond005/wav2vec2-large-ru-golos")

loading configuration file preprocessor_config.json from cache at /root/.cache/huggingface/hub/models--bond005--wav2vec2-large-ru-golos/snapshots/baa806852e641a9fa953ec1cc33de339ea02f9ae/preprocessor_config.json
Feature extractor Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": true,
  "sampling_rate": 16000
}

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bond005--wav2vec2-large-ru-golos/snapshots/baa806852e641a9fa953ec1cc33de339ea02f9ae/config.json
Model config Wav2Vec2Config {
  "_name_or_path": "bond005/wav2vec2-large-ru-golos",
  "activation_dropout": 0.0,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForCTC"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "classifier_proj_size": 256,

In [39]:
print(ds_train[0]['audio']["path"])

00003ec0-1257-42d1-b475-db1cd548092e.wav


In [40]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(ds_train)-1)

print(ds_train[rand_int]["transcription"])
ipd.Audio(data=ds_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)


мы в школе писали физру


In [41]:
def plot_waveform(waveform, sample_rate, title="Waveform", xlim=None):
    waveform = waveform.numpy()

    num_channels, num_frames = waveform.shape
    time_axis = torch.arange(0, num_frames) / sample_rate

    figure, axes = plt.subplots(num_channels, 1)
    if num_channels == 1:
        axes = [axes]
    for c in range(num_channels):
        axes[c].plot(time_axis, waveform[c], linewidth=1)
        axes[c].grid(True)
        if num_channels > 1:
            axes[c].set_ylabel(f"Channel {c+1}")
        if xlim:
            axes[c].set_xlim(xlim)
    figure.suptitle(title)
    plt.show(block=False)

In [42]:
def plot_specgram(waveform, sample_rate, title="Spectrogram", xlim=None):
    waveform = waveform.numpy()

    num_channels, _ = waveform.shape

    figure, axes = plt.subplots(num_channels, 1)
    if num_channels == 1:
        axes = [axes]
    for c in range(num_channels):
        axes[c].specgram(waveform[c], Fs=sample_rate)
        if num_channels > 1:
            axes[c].set_ylabel(f"Channel {c+1}")
        if xlim:
            axes[c].set_xlim(xlim)
    figure.suptitle(title)
    plt.show(block=False)

In [50]:
def aug_audio(SAMPLE_WAV):
    # Load the data
    waveform1 = SAMPLE_WAV.get('array')
    sample_rate1 = SAMPLE_WAV.get('sampling_rate')
    
    # Define effects
    effects = [
        ["lowpass", "-1", "300"],  # apply single-pole lowpass filter
        ["speed", "0.8"],  # reduce the speed
        # This only changes sample rate, so it is necessary to
        # add `rate` effect with original sample rate after this.
        ["rate", f"{sample_rate1}"],
        ["reverb", "-w"],  # Reverbration gives some dramatic feeling
    ]

    # Apply effects
    waveform2, sample_rate2 = torchaudio.sox_effects.apply_effects_tensor(waveform1, sample_rate1, effects)

    print(waveform1.shape, sample_rate1)
    print(waveform2.shape, sample_rate2)
    plot_waveform(waveform1, sample_rate1, title="Original", xlim=(-0.1, 3.2))
    plot_specgram(waveform1, sample_rate1, title="Original", xlim=(0, 3.04))
    Audio(waveform1, rate=sample_rate1)

In [51]:
rand_int = random.randint(0, len(ds_train)-1)
print(ds_train[rand_int]["transcription"])
aug_audio(ds_train[rand_int]["audio"])

так что вот эти все не знаю


RuntimeError: torchaudio::sox_effects_apply_effects_tensor() Expected a value of type 'Tensor' for argument '_0' but instead found type 'ndarray'.
Position: 0
Value: array([ 0.00015259,  0.00027466,  0.00045776, ..., -0.00479126,
       -0.00448608, -0.0050354 ])
Declaration: torchaudio::sox_effects_apply_effects_tensor(Tensor _0, int _1, str[][] _2, bool _3) -> (Tensor _0, int _1)
Cast error details: Unable to cast [ 0.00015259  0.00027466  0.00045776 ... -0.00479126 -0.00448608
 -0.0050354 ] to Tensor

In [13]:
ds_train

Dataset({
    features: ['audio', 'transcription'],
    num_rows: 5000
})

In [14]:
rand_int = random.randint(0, len(ds_train)-1)

print("Target text:", ds_train[rand_int]["transcription"])
print("Input array shape:", ds_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", ds_train[rand_int]["audio"]["sampling_rate"])


Target text: это не совсем правильно понял что именно я имел в виду тогда
Input array shape: (59200,)
Sampling rate: 16000


In [15]:
def prepare_dataset(batch):
    #print(batch)
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["transcription"]).input_ids
    return batch

ds_train = ds_train.map(prepare_dataset, remove_columns=ds_train.column_names)
print('train')
ds_test = ds_test.map(prepare_dataset, remove_columns=ds_test.column_names)
print('test')

Loading cached processed dataset at /root/.cache/huggingface/datasets/MegaKosT___parquet/MegaKosT--RuDevSberDS-c815bd8b35ec831d/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-3a7d74e9bafdf3e3.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/MegaKosT___parquet/MegaKosT--RuDevSberDS-c815bd8b35ec831d/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-bc472a0ba0177f94.arrow


train
test


In [16]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch


In [17]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [18]:
import datasets
wer_metric = datasets.load_metric("wer")

/tmp/ipykernel_1304/2798393314.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = datasets.load_metric("wer")


In [19]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


In [20]:
model.freeze_feature_encoder()

In [22]:
from transformers import TrainingArguments
from transformers import Trainer

training_args = TrainingArguments(
  output_dir='pisets/results',
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=True,
)

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=ds_train,
    eval_dataset=ds_test,
    tokenizer=processor.feature_extractor,
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/usr/src/app/pisets/pisets/results is already a clone of https://huggingface.co/doooori/results. Make sure you pull the latest changes with `repo.git_pull()`.
Using cuda_amp half precision backend
The following columns in the training set don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True

Step,Training Loss,Validation Loss,Wer
400,0.666300,0.407194,0.345260
800,0.532300,0.428288,0.365223
1200,0.393000,0.450940,0.353650
1600,0.316700,0.511819,0.377182
2000,0.252700,0.502363,0.360594
2400,0.208100,0.531912,0.359919
2800,0.180500,0.556832,0.348346
3200,0.149700,0.602786,0.354036
3600,0.127000,0.590052,0.342077


The following columns in the evaluation set don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1495
  Batch size = 8
Saving model checkpoint to pisets/results/checkpoint-400
Configuration saved in pisets/results/checkpoint-400/config.json
Model weights saved in pisets/results/checkpoint-400/pytorch_model.bin
Feature extractor saved in pisets/results/checkpoint-400/preprocessor_config.json
Feature extractor saved in pisets/results/preprocessor_config.json
/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate ca

KeyboardInterrupt: 